In [1]:
import os, tqdm, glob, pickle,re
import pandas as pd

# from konlpy.tag import Twitter,Kkma,Komoran,Hannanum
import soynlp
from soynlp.word import WordExtractor
from soynlp.noun import LRNounExtractor
from soynlp.tokenizer import LTokenizer,MaxLRScoreTokenizer, MaxScoreTokenizer
import nltk

# from gensim.models import Word2Vec
# from sklearn.manifold import TSNE

import matplotlib as mpl
import matplotlib.font_manager as fm

import matplotlib.pyplot as plt
plt.style.use('ggplot')

from collections import Counter
# from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
plt.rcParams["font.family"] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.size'] = 24.
plt.rcParams['xtick.labelsize'] = 20.
plt.rcParams['ytick.labelsize'] = 20.
plt.rcParams['axes.labelsize'] = 20.

In [6]:
# os.chdir('./data')

In [17]:
# SUV로 쿼리한 문서
with open('suv_docs.lst','rb') as fp:
    all_docs = pickle.load(fp)

In [18]:
all_docs[2]

{'board_cat': 'consumer',
 'content': '헤치백 아니죠... SUV 입니다!',
 'doc_id': '592147b4c87a569be00376c5',
 'lang': 'KOR',
 'post_dt': '2016-05-03T21:20:00Z',
 'site_nm': '[네이버카페]니로공식동호회',
 'title': 'SUV다!'}

pandas 로 변환

In [19]:
doc_data = {'doc_id':[],'site_nm':[],'board_cat':[],'post_dt':[],'title':[], 'content':[]}

In [20]:
for doc in all_docs:
    doc_data['doc_id'].append(doc['doc_id'])
    doc_data['site_nm'].append(doc['site_nm'])
    doc_data['board_cat'].append(doc['board_cat'])
    doc_data['post_dt'].append(doc['post_dt'])
    doc_data['title'].append(doc['title'])
    doc_data['content'].append(doc['content'])

In [21]:
doc_df = pd.DataFrame(doc_data)

In [22]:
del all_docs,doc_data

In [23]:
to_drop = doc_df[doc_df.duplicated(subset='doc_id')].index

In [24]:
doc_df.drop(index=to_drop,inplace=True)

In [25]:
doc_df.reset_index(drop=True,inplace=True)

In [26]:
doc_df.describe(include='object')

,doc_id,site_nm,board_cat,post_dt,title,content
count,95133,95133,95133,95133,95133,95133
unique,95133,512,1,78143,84467,86339
top,59cdac618a6a6924984c040b,보배드림,consumer,2016-10-16T15:00:00Z,안녕하세요,[THE SUV SPORTAGE] 클럽QL 더많은 정보와 즐거운 커뮤니티공간 !
freq,1,13175,95133,27,36,152


In [27]:
doc_df.sample(2)

,doc_id,site_nm,board_cat,post_dt,title,content
55939,5a4aafc6eca98518e8b08ca1,싼타페DM 공식동호회 ★디엠러브★ 싼타페TM 산타페 더 프라임,consumer,2018-01-01T11:47:00Z,중고차 구매.. 조언 좀 구합니다^^,올해 첫째아기가 태어나기에 현재 경차타고 있는데 suv로 바꾸기로 마음먹고 투싼이나...
71397,59cce312d60e3c2df109e734,보배드림,consumer,2017-01-30T08:09:00Z,새로나온 모하비 시승해보니..ㅎ,아쉬운건 몇개있네요 물론 지금 타고있는 신형 k7디제루 대비.... 1. 오토홀드 ...


In [28]:
doc_df.site_nm.value_counts()[:5]

보배드림        13175
뽐뿌           6230
디시인사이드       5351
일간베스트저장소     3528
클리앙          3049
Name: site_nm, dtype: int64

In [29]:
doc_df = doc_df[pd.Series(map(len,doc_df.content))<5000] # 지나치게 긴글 제외
doc_df.reset_index(drop=True,inplace=True)
doc_df.describe(include='object')

,doc_id,site_nm,board_cat,post_dt,title,content
count,94327,94327,94327,94327,94327,94327
unique,94327,512,1,77564,83928,85728
top,59cdac618a6a6924984c040b,보배드림,consumer,2016-10-16T15:00:00Z,안녕하세요,[THE SUV SPORTAGE] 클럽QL 더많은 정보와 즐거운 커뮤니티공간 !
freq,1,13047,94327,27,36,152


In [22]:
docs = []
for i in range(len(doc_df)):
    docs.append(doc_df.loc[i,'title']+'. '+doc_df.loc[i,'content'])

In [23]:
# 코퍼스 생성
corpus = []
bmarks = [0,]
bmark = 0
prog = 0
for i,doc in enumerate(docs):
#     if prog%1000 == 0:
#         print(prog)
    prog += 1
    sents = nltk.sent_tokenize(doc)
    bmark += len(sents)
    bmarks.append(bmark)
    
    docs[i] = [sent for sent in sents if len(sent)>4]
    corpus += [sent for sent in sents if len(sent)>4]

In [53]:
docs[0]

['차량제원표입니다~(준중형SUV,중형세단,중형SUV,대형SUV,그랜저).',
 '비 고 소나타 K5 말리부 SM6 비 고 그랜저 전장(길이) 4,855 4,855 4,925 4,850 전장(길이) 4,930 전폭(넓이) 1,865 1,860 1,855 1,870 전폭(넓이) 1,865 전고(높이) 1,475 1,475 1,470 1,460 전고(높이) 1,470 축거 2,805 2,805 2,830 2,810 축거 2,845 비 고 QM6 싼타페 쏘렌토 캡티바 이쿼녹스 전장(길이) 4,675 4,770 4,800 4,690 4,650 전폭(넓이) 1,845 1,890 1,890 1,850 1,845 전고(높이) 1,680 1,680~1,705 1,690 1,725~1,755 1,690~1,695 축거 2,705 2,765 2,780 2,705 2,725 비 고 투싼 스포티지 코란도C 전장(길이) 4,480 4485~4495 4,410 전폭(넓이) 1,850 1,855 1,830 전고(높이) 1,645~1,650 1,635~1,645 1,715 축거 2,670 2,670 2,650 비 고 G4렉스턴 모하비 전장(길이) 4,850 4,930 전폭(넓이) 1,960 1,915 전고(높이) 1,825 1,810 축거 2,865 2,895 차량별 제원입니다.',
 '정말 무쟈게 잉여해서 만들어봤습니다.',
 '가끔 논란이되는 QM6의 중형?준중형?',
 '도 궁금할뿐더러 오늘 핫이슈였던 과연 그랜저와 말리부는?',
 '이라는 물음이 들어서.ㅎㅎ QM6가 투싼급이 맞다면 그랜저와 말리부는 동급이다라고 주장하는 분들의 의견도 어느정도 아예 일리없는 주장은 아니지않나 하는 생각이 들정도로 제원차이는 별 차이가 없군요(물론제원이 다는 아니지만^^;).... 아!',
 '물론 전 QM6는 중형, 그랜저는 준대형이라고 생각합니다.!',
 '출처는 네이버이고 모델은 가장최신형(19년or18년형)으로 작성하였습니다.']

In [52]:
corpus[0]

'차량제원표입니다~(준중형SUV,중형세단,중형SUV,대형SUV,그랜저).'

soynlp 라이브러리 이용해 코퍼스에서 명사와 word 각각 추출

https://github.com/lovit/soynlp


In [25]:
noun_extractor = LRNounExtractor(
    max_left_length=10,
    max_right_length=7,
    predictor_fnames=None,
    verbose=True

)

used default noun predictor; Sejong corpus predictor
used noun_predictor_sejong
All 2398 r features was loaded


In [26]:
nouns = noun_extractor.train_extract(
    corpus,
    min_noun_score=0.3,
    min_noun_frequency=20
)

scanning completed
(L,R) has (129998, 73868) tokens
building lr-graph completed

In [27]:
with open('nouns.pickle','wb') as fp:
    pickle.dump(nouns,fp)

In [59]:
word_extractor = WordExtractor(
    min_frequency=10,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)

In [60]:
word_extractor.train(corpus)

training was done. used memory 2.118 Gbse memory 2.387 Gb


In [61]:
word_scores = word_extractor.extract()

all cohesion probabilities was computed. # words = 249258
all branching entropies was computed # words = 744809
all accessor variety was computed # words = 744809


In [74]:
with open('word_scores_suv.pickle','wb') as fp:
    pickle.dump(word_scores,fp)

In [62]:
word_scores['펠리세이드']

Scores(cohesion_forward=0.9126350408407692, cohesion_backward=0.14893782726600285, left_branching_entropy=3.1886986358729428, right_branching_entropy=3.6009987062542095, left_accessor_variety=67, right_accessor_variety=72, leftside_frequency=752, rightside_frequency=19)

WordExtractor가 계산하는 것은 다양한 종류의 단어 가능 점수들입니다. 이를 잘 조합하여 원하는 점수를 만들 수도 있습니다. 

즐겨쓰는 방법 중 하나는 cohesion_forward에 right_branching_entropy를 곱하는 것으로, 

*(1) 주어진 글자가 유기적으로 연결되어 함께 자주 나타나고, *

*(2) 그 단어의 우측에 다양한 조사, 어미, 혹은 다른 단어가 등장하여 단어의 우측의 branching entropy가 높다는 의미입니다.*

In [63]:
import math

def word_score(score):
    return (score.cohesion_forward * math.exp(score.right_branching_entropy))

print('단어   (빈도수, cohesion, branching entropy)\n')
for word, score in sorted(word_scores.items(), key=lambda x:word_score(x[1]), reverse=True)[:30]:
    print('%s     (%d, %.3f, %.3f)' % (
            word, 
            score.leftside_frequency, 
            score.cohesion_forward,
            score.right_branching_entropy
            )
         )

단어   (빈도수, cohesion, branching entropy)

..     (5746, 0.906, 5.399)
함께     (6962, 0.887, 5.264)
^^     (3661, 0.938, 5.191)
;;     (492, 0.934, 5.109)
으로     (717, 0.708, 5.385)
합니다.     (11336, 0.832, 5.212)
...     (3091, 0.698, 5.372)
됩니다.     (2476, 0.918, 5.059)
때문에     (6973, 0.674, 5.367)
입니다.     (5622, 0.741, 5.250)
ㅠㅠ     (4870, 0.836, 5.089)
었습니다.     (11, 0.785, 5.144)
ㅜㅜ     (1694, 0.830, 5.046)
너무     (12416, 0.926, 4.922)
봅니다.     (1127, 0.886, 4.915)
ㅎㅎ     (6404, 0.916, 4.825)
통해     (10994, 0.668, 5.125)
suv     (36311, 0.913, 4.730)
께서     (30, 0.789, 4.874)
였습니다.     (263, 0.747, 4.921)
까지     (1339, 0.432, 5.458)
했습니다.     (1924, 0.595, 5.132)
겠습니다.     (53, 0.617, 5.091)
SUV     (110155, 0.922, 4.682)
~~     (428, 0.588, 5.125)
....     (853, 0.512, 5.262)
갑자기     (1474, 0.737, 4.896)
역시     (8064, 0.538, 5.201)
근데     (4382, 0.655, 5.002)
줍니다.     (243, 0.947, 4.632)


토크나이저 생성

In [64]:
cohesion_scores = {word:score.cohesion_forward for word, score in word_scores.items()}
ltokenizer = LTokenizer(scores = cohesion_scores)
mtokenizer = MaxScoreTokenizer(scores = cohesion_scores)

In [72]:
with open('cohesion_scores_suv.pickle','wb') as fp:
    pickle.dump(cohesion_scores,fp)

In [65]:
mtokenizer

In [31]:
ltokenizer(corpus[3])

['가끔', '논란이되는', 'QM6의', '중형?준중형?']

In [32]:
mtokenizer(corpus[3])

['가끔', '논란이되는', 'QM6의', '중형?준중형?']

In [33]:
ltokenizer(corpus[8])

['가솔린', '세단>>>>>>>>>>>디젤suv>>>>가솔린suv>>>>>>>>>>>>>>>>>>>>>>>디젤세단.']

In [34]:
mtokenizer(corpus[8])

['가솔린',
 '세단',
 '>>>>>>>>>>',
 '>',
 '디젤',
 'suv',
 '>>>>',
 '가솔린',
 'suv',
 '>>>>>>>>>>',
 '>>>>>>>>>>',
 '>>>',
 '디젤',
 '세단',
 '.']